In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import Document, SystemMessage, HumanMessage

Initializing the GROQ API key that is stored in.env file

In [2]:
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')

Setting the LLM model which we want to use

In [3]:
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

Initializing embeddings and model which we want to use

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
comet_ml is installed but `COMET_API_KEY` is not set.


Specifying the directory where vector database is stored

In [5]:
persist_directory = "../RAG_3_vectordb_3_separate codes/entities_chroma_db"

Checking if the directory exists

In [6]:
if not os.path.exists(persist_directory):
    print("Persist directory does not exist.")
else:
    print("Persist directory exists.")

Persist directory exists.


Loading already created vector database. By specifying the embedding function, we are ensuring that the same model used to create the database is being used to query it.

In [7]:
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

Providing additional information about the metadata for more precise filtering and details about the document description helps the chain understand the document's content.

In [8]:
metadata_field_info = [
    AttributeInfo(
        name="year",
        description="Year the paper was published",
        type="integer",
    ),
    AttributeInfo(
        name="title",
        description="Title of the paper",
        type="string",
    ),
    AttributeInfo(
        name="last_section_title",
        description="Title section is associated with paragraph",
        type="string or list[string]",
    )
]

document_content_description = "Provides context of each paragraph within the article"

Setting up the SelfQueryRetriever that is used for retriving documents from the vector database. In this chain we specify that we are using Llama 3 as our LLM model, which sees analyzes the prompt and structures it into a query, entities_chroma_db as our vector database and provide also the additional attribute info about metadata and document description.

In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

Creating a custom prompt instructs the model to always respond in full sentences and to say "I don't know" if it doesn't know the answer. This approach prevents Llama 3 from generating random responses simply to fulfill the expectation of an answer.

In [11]:
custom_prompt_template = """Use the following pieces of information to answer the user's question. Always answear the question as if you were a human and answear in full sentance. During your answear be really specific. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [12]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()

Here, we define the complete question-answering chain in RAG. We specify that Llama 3 is the LLM model in use, the retriever is the SelfQueryRetriever we created earlier, chain_type="stuff" indicates a basic chain where the LLM answers based solely on the user's question and retrieved documents, and finally, we ensure that the custom prompt we created is used.

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

Testing part

In [14]:
query = "How was study conduted in article: A Multilevel Model of Resistance to Information Technology Implementation, be very specififc"
result = qa({"query": query})
print("Answer:", result["result"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer: According to the article "A Multilevel Model of Resistance to Information Technology Implementation", the study was conducted through a survey of 246 employees from 44 teams in a large financial institution in the United States, where participants were asked to complete a questionnaire assessing their perceptions of the technology, their attitudes towards its implementation, and their behavioral intentions regarding its use.


In [15]:
query = "How was study conduted in article: The Integrative Framework of Technology Use: An Extension and Test."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "The Integrative Framework of Technology Use: An Extension and Test" was conducted through a survey of 1,467 employees from 82 organizations, where participants were asked to complete an online questionnaire that assessed their technology use, perceived usefulness, ease of use, and other relevant factors.


In [16]:
query = "Is survey used in article: Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation" does indeed utilize a survey as a research methodology to collect data from participants, which is then used to analyze and predict various conceptualizations of system use.


In [17]:
query = "How is data analysed in article: Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation", data analysis was performed using a two-step approach, where first, a confirmatory factor analysis (CFA) was conducted to validate the measurement model, and then, a structural equation modeling (SEM) analysis was employed to examine the relationships between the constructs, specifically behavioral intention, facilitating conditions, and behavioral expectation, in predicting system use.


Testing part

In [18]:
query = "How was the study conducted in the article: Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance", the study was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically using a combination of surveys, interviews, and content analysis of team communication data to examine the impact of automaticity and communication structuration on virtual team behavior and performance.


In [19]:
query = "How was the study conducted in the article: Understanding User Responses to Information Technology: A Coping Model of User Adaptation."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "Understanding User Responses to Information Technology: A Coping Model of User Adaptation" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 120 employees who had recently undergone a technology change, as well as 30 in-depth interviews with a subset of these employees to gather more detailed insights into their experiences and coping mechanisms.


In [20]:
query = "How was the study conducted in the article: Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 154 employees from 12 organizations, as well as 36 in-depth interviews with employees and managers from these organizations, to gain a comprehensive understanding of system usage at both individual and organizational levels.


In [21]:
query = "How was the study conducted in the article: The Integrative Framework of Technology Use: An Extension and Test."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "The Integrative Framework of Technology Use: An Extension and Test" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 1,433 employees from 12 organizations, as well as 30 in-depth interviews with employees and managers to gather more nuanced insights into technology use in the workplace.


In [22]:
query = "How was the study conducted in the article: A Multilevel Model of Resistance to Information Technology Implementation."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "A Multilevel Model of Resistance to Information Technology Implementation", the study was conducted through a survey of 357 employees from 24 business units in a large financial institution, where the researchers collected data on individual, group, and organizational factors that influence resistance to IT implementation, and then analyzed the data using hierarchical linear modeling to examine the relationships between these factors and resistance to IT implementation.


In [23]:
query = "How was the study conducted in the article: When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 120 employees from two organizations, followed by 30 in-depth interviews, and then a statistical analysis of the survey data to identify patterns and correlations between technology use and network change.


In [24]:
query = "How was the study conducted in the article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance", the study was conducted through a survey of 155 employees who used a specific information system for their daily work, and the researchers collected data at three points in time, with a three-month interval between each data collection point, to examine the relationships between intention, habit, and continuance behavior.


In [25]:
query = "How was the study conducted in the article: A Temporally Situated Self-Agency Theory of Information Technology Reinvention."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, I don't have access to the specific article "A Temporally Situated Self-Agency Theory of Information Technology Reinvention" and therefore, I don't know how the study was conducted in that particular article.


In [26]:
query = "How was the study conducted in the article: Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use", the study was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically using a survey of 324 participants and 30 in-depth interviews to gather data on individuals' habits and emotions related to information technology use.


In [27]:
query = "How was the study conducted in the article: An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance", the study was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, where the researchers collected data from 120 employees across 12 organizations over a period of 6 months, using a combination of surveys, interviews, and system log data to examine the patterns of information systems use and their impact on employee performance.


In [28]:
query = "How was the study conducted in the article: The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption", the study was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 215 employees from 12 organizations, as well as 36 in-depth interviews with employees and managers from these organizations, to explore the habits and routines related to information systems use at both individual and organizational levels.


In [29]:
query = "How was the study conducted in the article: Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: According to the article "Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis", the study was conducted through a survey of 215 customer service managers from various industries, where they were asked to provide information about their organization's customer service process, IT capabilities, and performance outcomes, and the data was then analyzed using structural equation modeling to examine the relationships between IT, customer service process, and performance.


In [30]:
query = "How was the study conducted in the article: An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, I don't have access to the specific article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" and therefore, I don't know how the study was conducted in that particular article.


In [31]:
query = "How was the study conducted in the article: Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, I don't have access to the specific article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations" and therefore, I don't know how the study was conducted in that particular article.


In [32]:
query = "How was the study conducted in the article: Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns" was conducted through a qualitative research approach, specifically using a grounded theory methodology, which involved in-depth interviews with 23 individuals who were experiencing difficulties with a new enterprise system implementation in their organization.


In [33]:
query = "How was the study conducted in the article: How Do Suppliers Benefit from Information Technology Use in Supply Chain Relationships?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study was conducted through a survey of 157 suppliers in the United States, where they were asked to provide information about their use of information technology, such as electronic data interchange (EDI) and enterprise resource planning (ERP) systems, as well as their perceptions of the benefits they received from using these technologies in their supply chain relationships.


In [34]:
query = "How was the study conducted in the article: Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The study in the article "Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 215 participants and a subsequent in-depth interview with 30 of these participants to gather more detailed insights into their experiences and behaviors when using information system features.


In [35]:
query = "How was the study conducted in the article: Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, I don't have enough information to provide a specific answer to how the study was conducted in the article "Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation" as the context provided is limited.


In [36]:
query = "How was the study conducted in the article: A Dramaturgical Model of the Production of Performance Data."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, I don't have access to the article "A Dramaturgical Model of the Production of Performance Data" and therefore, I don't know how the study was conducted in that specific article.


In [37]:
query = "What is the theory in the article: Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The theory in the article "Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance" is Structuration Theory, which suggests that human behavior is shaped by the interplay between the social structure and the individual's actions, and that communication plays a crucial role in this process.


In [38]:
query = "What is the theory in the article: Understanding User Responses to Information Technology: A Coping Model of User Adaptation"
result = qa({"query": query})
print("Answer:", result["result"])




Answer: The theory in the article "Understanding User Responses to Information Technology: A Coping Model of User Adaptation" is the Coping Model of User Adaptation, which proposes that users respond to information technology in three stages: initial resistance, coping, and adaptation, as they attempt to manage the stress and uncertainty associated with the introduction of new technology.


In [39]:
query = "What is the theory in the article: Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The theory presented in the article "Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective" is the Unified Theory of Acceptance and Use of Technology (UTAUT), which aims to explain how users form attitudes and intentions to use technology, and how these attitudes and intentions influence actual system usage behavior within organizational settings.


In [40]:
query = "What is the theory in the article: The Integrative Framework of Technology Use: An Extension and Test"
result = qa({"query": query})
print("Answer:", result["result"])



Answer: The theory in the article "The Integrative Framework of Technology Use: An Extension and Test" is the Unified Theory of Acceptance and Use of Technology (UTAUT).


In [41]:
query = "What is the theory in the article: A Multilevel Model of Resistance to Information Technology Implementation"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "A Multilevel Model of Resistance to Information Technology Implementation" is the Unified Theory of Acceptance and Use of Technology (UTAUT), which explains how users form attitudes and intentions to use technology, and how these attitudes and intentions influence their actual behavior.


In [42]:
query = "What is the theory in the article: When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The theory in the article "When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances" is the Affordance Theory, which suggests that the functionality of a technology, or its affordances, influence the behavior and interactions of individuals and groups within an organization, ultimately enabling or constraining network change.


In [43]:
query = "What is the contribution for our understanding of IT in the article: When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances" contributes to our understanding of IT by highlighting the importance of shared affordances, which are the possibilities for action provided by a technology, in enabling network change in organizations, and how this differs from mere feature use, which is the actual utilization of a technology's features.


In [44]:
query = "What is the contribution for our understanding of IT in the article: A Multilevel Model of Resistance to Information Technology Implementation"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "A Multilevel Model of Resistance to Information Technology Implementation" contributes to our understanding of IT by providing a comprehensive framework that explains the various factors that influence an individual's resistance to IT implementation, including individual, group, and organizational levels, thereby helping to identify and address potential barriers to successful IT adoption.


In [45]:
query = "What is the contribution for our understanding of IT in the article: Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective" contributes to our understanding of IT by highlighting the importance of considering multiple levels of analysis, including individual, group, and organizational levels, when examining system usage in organizations, and by emphasizing the need to move beyond a sole focus on individual-level factors to understand how IT systems are used in organizational contexts.


In [46]:
query = "What is the contribution for our understanding of IT in the article: Understanding User Responses to Information Technology: A Coping Model of User Adaptation"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Understanding User Responses to Information Technology: A Coping Model of User Adaptation" contributes to our understanding of IT by proposing a coping model of user adaptation, which highlights the importance of understanding how users respond to IT in terms of their emotional, cognitive, and behavioral reactions, and how these reactions influence their ability to adapt to new technologies.


In [47]:
query = "What is the contribution for our understanding of IT in the article: Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance" contributes to our understanding of IT by highlighting the importance of structuration of communication in virtual teams, which is facilitated by information technology, and how it influences team behavior and performance, thereby shedding light on the role of IT in shaping the dynamics of virtual teams.


In [48]:
query = "What is the contribution for our understanding of IT in the article: When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances" contributes to our understanding of IT by highlighting the importance of shared affordances, which are the possibilities for action provided by a technology, in enabling network change in organizations, and how this differs from mere feature use, which is the actual utilization of a technology's features.


In [49]:
query = "What was the empirical approach for the article: A Multilevel Model of Resistance to Information Technology Implementation"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "A Multilevel Model of Resistance to Information Technology Implementation" was a survey-based study, where the researchers collected data through a questionnaire administered to 202 employees from 14 organizations that had recently implemented a new information technology system.


In [50]:
query = "What was the empirical approach for the article: When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances" was a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 120 employees in two organizations, followed by semi-structured interviews with 30 of these employees, and finally, a social network analysis of their communication patterns.


In [51]:
query = "What was the empirical approach for the article: Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance" was a mixed-methods approach, combining both qualitative and quantitative methods, specifically using surveys, interviews, and content analysis to collect and analyze data from 120 virtual team members across 30 teams.


In [52]:
query = "What was the empirical approach for the article: Understanding User Responses to Information Technology: A Coping Model of User Adaptationn"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "Understanding User Responses to Information Technology: A Coping Model of User Adaptation" was a survey-based study, where the researchers collected data through a questionnaire administered to 134 employees who were in the process of adapting to a new technology in their workplace.


In [53]:
query = "What was the empirical approach for the article: Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective" was a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 154 employees from 12 organizations, as well as 36 semi-structured interviews with employees and managers from these organizations.


In [54]:
query = "What was the empirical approach for the article: The Integrative Framework of Technology Use: An Extension and Test"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "The Integrative Framework of Technology Use: An Extension and Test" was a survey research design, which involved collecting data through a questionnaire administered to a sample of 1,467 employees from various organizations.


qualitative prompt : theory

In [64]:
query = "What is the theory in the article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance" is the Theory of Planned Behavior, which is extended by incorporating the concept of habit to explain the continuance of information systems.


In [65]:
query = "What is the theory in the article: A Temporally Situated Self-Agency Theory of Information Technology Reinvention"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "A Temporally Situated Self-Agency Theory of Information Technology Reinvention" is a theoretical framework that explains how individuals reinvent information technology through a process of self-reflection, agency, and temporal situatedness, suggesting that people's perceptions of themselves and their relationship with technology change over time, influencing their reinvention of technology.


In [66]:
query = "What is the theory in the article: Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use" is the Theory of Planned Behavior, which suggests that an individual's intention to perform a behavior is influenced by their attitude towards the behavior, subjective norms, and perceived behavioral control.


In [67]:
query = "What is the theory in the article: An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance" is the Theory of Planned Behavior, which suggests that an individual's behavior is influenced by their attitudes, subjective norms, and perceived behavioral control.


In [68]:
query = "What is the theory in the article: The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption" is Institutional Theory, which suggests that organizational habits and routines are shaped by institutional pressures and become embedded in the organization over time, and that these habits and routines can be disrupted by changes in the institutional environment.


In [69]:
query = "What is the theory in the article: Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The theory in the article "Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis" is the Resource-Based View (RBV) theory, which suggests that a firm's performance is determined by its unique resources and capabilities, and how they are utilized to create a competitive advantage.


qualitative prompt : Contribution to our understanding of IT use

In [70]:
query = "What is the contribution for our understanding of IT in the article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance" contributes to our understanding of IT by highlighting the significant role of habit in shaping individuals' continued use of information systems, and how this habit can limit the predictive power of intention, which is a crucial aspect of IT adoption and continuance.


In [71]:
query = "What is the contribution for our understanding of IT in the article: A Temporally Situated Self-Agency Theory of Information Technology Reinvention"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "A Temporally Situated Self-Agency Theory of Information Technology Reinvention" contributes to our understanding of IT by introducing a novel theoretical framework that highlights the role of human agency and temporal situatedness in shaping the reinvention of information technology, emphasizing that IT is not just a passive tool, but an active participant in the process of organizational change and innovation.


In [72]:
query = "What is the contribution for our understanding of IT in the article:Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use" contributes to our understanding of IT by highlighting the significance of habits and emotions in shaping individuals' continued use of information technology, and challenging the traditional view that intentions are the primary driver of IT adoption and continued use.


In [73]:
query = "What is the contribution for our understanding of IT in the article:An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance" contributes to our understanding of IT by highlighting the significance of technological events as triggers that influence information systems use patterns, and how these patterns change over time, ultimately affecting individual performance.


In [74]:
query = "What is the contribution for our understanding of IT in the article:The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption" contributes to our understanding of IT by highlighting how information systems habits are deeply embedded in organizational and individual level routines, and how these habits can both facilitate and hinder the adoption and use of new information systems, thereby providing insights into the complexities of IT adoption and implementation.


In [75]:
query = "What is the contribution for our understanding of IT in the article:Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The article "Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis" contributes to our understanding of IT by highlighting its role as a key resource that can enhance the performance of customer service processes, particularly by improving the efficiency and effectiveness of customer service operations, and by providing a framework for understanding how IT can be leveraged to create competitive advantages in customer service.


qualitative prompt : Empirical approach

In [78]:
query = "What was the empirical approach for the article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The empirical approach used in the article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance" was a survey-based study, specifically a longitudinal survey design, where the researchers collected data from 155 employees over a period of three months, using a combination of self-reported measures and objective system usage data to examine the relationship between habit, intention, and continuance behavior.


In [77]:
query = "What was the empirical approach for the article: A Temporally Situated Self-Agency Theory of Information Technology Reinvention"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The empirical approach used in the article "A Temporally Situated Self-Agency Theory of Information Technology Reinvention" was a longitudinal case study, which involved collecting and analyzing data from a single organization over a period of 18 months, using a combination of interviews, observations, and archival data.


In [80]:
query = "What was the empirical approach for the article: Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach used in the article "Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use" was a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a survey of 145 participants and 30 in-depth interviews to gather data on individuals' experiences and behaviors related to information technology use.


In [81]:
query = "What was the empirical approach for the article:  An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The empirical approach used in the article "An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance" was a longitudinal field study, which involved collecting data from 114 employees over a period of 12 weeks, using a combination of surveys, system logs, and performance metrics to examine the patterns of information systems use and their consequences for performance.


In [82]:
query = "What was the empirical approach for the article: The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The empirical approach used in the article "The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption" was a mixed-methods approach, combining both qualitative and quantitative methods, specifically involving a longitudinal case study of a large organization over a period of 18 months, which included 35 semi-structured interviews, observations, and archival data analysis.


In [92]:
query = "What was the empirical approach for the article: Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The empirical approach used in the article "Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis" was a survey-based study, where data was collected from 215 customer service managers in the banking industry through a self-administered questionnaire.


qualitative prompt : How use was studied or conceptualized

In [85]:
query = "How was study conducted or conceptualized in article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance" was conducted through a survey of 155 employees who used a specific information system at their workplace, and the data was collected at three points in time, with a three-month interval between each data collection point, allowing the researchers to examine the relationships between intention, habit, and continuance behavior over time.


In [86]:
query = "How was study conducted or conceptualized in article: A Temporally Situated Self-Agency Theory of Information Technology Reinvention"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "A Temporally Situated Self-Agency Theory of Information Technology Reinvention" was conceptualized through a qualitative research approach, specifically using a longitudinal case study design, where the authors collected and analyzed data from 24 interviews with 12 individuals over a period of 18 months, examining how they reinvented information technology in their daily work practices.


In [87]:
query = "How was study conducted or conceptualized in article: Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically using a survey of 324 participants and 30 in-depth interviews to explore the complex relationships between intention, habit, and emotion in shaping individuals' continued use of information technology.


In [88]:
query = "How was study conducted or conceptualized in article:  An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically using a survey questionnaire to collect data from 215 employees in a large organization, and then analyzing the data using statistical models to identify patterns and relationships between technological events, time, and performance.


In [89]:
query = "How was study conducted or conceptualized in article: The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption" was conducted through a mixed-methods approach, combining both qualitative and quantitative data collection and analysis methods, specifically involving a longitudinal case study of a large organization over a period of two years, including 43 semi-structured interviews with employees and managers, as well as surveys and observational data.


In [90]:
query = "How was study conducted or conceptualized in article: Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis"
result = qa({"query": query})
print("Answer:", result["result"])


Answer: The study in the article "Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis" was conducted through a survey of 215 customer service managers from various industries, where they were asked to provide information about their organization's customer service process, IT capabilities, and performance outcomes, and the data was then analyzed using structural equation modeling to examine the relationships between IT, customer service process, and performance.
